# Equivalent Water Thickness/Canopy Water Content from Imaging Spectroscopy Data

#TODO - Choose Study Site(s), Sedgwick Reserve is just an example.
#TODO - Should we refer to this as EWT or CWC?

**Summary**  

In this notebook we will explore how Canopy Water Content (CWC) or Equivalent Water Thickness (EWT) can be calculated from the Earth Surface Mineral Dust Source Investigation (EMIT) L2A Reflectance Product, then we will apply this knowledge to calculate CWC over the [Sedgewick Reserve](https://nrs.ucsb.edu/sedgwick-reserve/) located in the Santa Ynez Valley approximately 35 miles north of Santa Barbara, CA. 

**Background**

Surface reflectance is the fraction of incoming solar radiation reflected Earth's surface. Materials reflect proportions of radiation differently based upon their chemical composition and physical properties, giving materials their own unique spectral signature or fingerprint. This means that reflectance information can be used to determine the composition information about a target, including water content. For the purposes of this example we will use CWC and EWT interchangeably as the predicted thickness or absorption path length in centimeters (cm) of water that would be required to yield an observed spectra, or the water content of the canopy (g/cm^2).  These properties are an indicator of vegetation type and health as well as wildfire risk. The methods used here to calculate CWC are based on the [ISOFIT python package](https://github.com/isofit/isofit/tree/main). The Beer-Lambert physical model used to calculate CWC is described in [Green et al. (2006)](https://doi.org/10.1029/2005WR004509) and [Bohn et al. (2020)](https://doi.org/10.1016/j.rse.2020.111708). Of note, this model does not account for multiple scattering effects within the canopy and may result in oversestimation of CWC (Bohn et al., 2020).

The [Sedgewick Reserve](https://nrs.ucsb.edu/sedgwick-reserve/) contains a diverse assortment of vegetation including coast live oak forest, blue oak woodland, valley oak savannah, buckbrush chaparral, coastal sage scrub, grassland, willow riparian forest, serpentine outcroppings, and agricultural lands.

More about the [EMIT mission] and [EMIT products].

**References**

- Shrestha, Rupesh. 2023. Equivalent water thickness/canopy water content from hyperspectral data. Jupyter Notebook. Oak Ridge National Laboratory Distributed Active Archive Center. https://github.com/rupesh2/ewt_cwc/tree/main
- Bohn, N., L. Guanter, T. Kuester, R. Preusker, and K. Segl. 2020. Coupled retrieval of the three phases of water from spaceborne imaging spectroscopy measurements. Remote Sensing of Environment 242:111708. https://doi.org/10.1016/j.rse.2020.111708
- Green, R.O., T.H. Painter, D.A. Roberts, and J. Dozier. 2006. Measuring the expressed abundance of the three phases of water with an imaging spectrometer over melting snow. Water Resources Research 42:W10402. https://doi.org/10.1029/2005WR004509
- Thompson, D.R., V. Natraj, R.O. Green, M.C. Helmlinger, B.-C. Gao, and M.L. Eastwood. 2018. Optimal estimation for imaging spectrometer atmospheric correction. Remote Sensing of Environment 216:355–373. https://doi.org/10.1016/j.rse.2018.07.003

**Requirements** 
 - None for Workshop!
 - To set up Python Environment locally - See **setup_instructions.md** in the `/setup/` folder

**Learning Objectives**  
- Calculate the EWT of a single pixel
- Calculate the EWT of a ROI

**Tutorial Outline**  

1.1 Setup  
1.2 Orthorectification
1.3 Extracting A Reflectance Pixel
1.4 Beer-Lambert Law  
1.5 Inversion  
1.5 EWT/CWC  
1.6 Clipping to a Polygon  
1.7 Applying Inversion in Parallel Across an ROI

In [ ]:
# Install ray on 2i2c
#!pip install "ray[default]"
!pip install openpyxl

# Import Python Libraries

In [ ]:
# Import Packages
import os
import glob
import earthaccess
import math
import numpy as np
import xarray as xr
from osgeo import gdal
import rasterio as rio
import rioxarray as rxr
from matplotlib import pyplot as plt
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import pandas as pd
import geopandas as gp
import sys
from modules.emit_tools import emit_xarray, ortho_xr
from modules.ewt_calc import calc_ewt
from scipy.optimize import least_squares

## Open an EMIT File

#TODO - Some sort of text

Set a filepath for a downloaded EMIT file, and open using the `emit_xarray` function.


In [ ]:
fp = '../data/EMIT_L2A_RFL_001_20230405T190311_2309513_002.nc'

In [ ]:
ds = emit_xarray(fp,ortho=True)

We can retrieve the spectra from a sample point by providing a latitude and longitude along with a method the `sel` function. This will select the pixel closest to the provided coordinates.

In [ ]:
point = ds.sel(latitude=34.568,longitude=-120.043, method='nearest')
point

We can plot this information to see the spectra.

In [ ]:
point.hvplot.line(x='wavelengths',y='reflectance',color='black').opts(title=f"Latitude: {point.latitude.values:.3f} Longitude: {point.longitude.values:.3f}")

## Calculating CWC

Estimation of the CWC/EWT is done using by solving a nonlinear least squares function to minimize the residual of a Beer-Lambert model. 
First, we want to define a function that returns the vector of residuals between measured and modeled surface reflectance

In [ ]:
# https://github.com/isofit/isofit/blob/main/isofit/inversion/inverse_simple.py
def beer_lambert_model(x, y, wl, alpha_lw):
    """Function, which computes the vector of residuals between measured and modeled surface reflectance optimizing
    for path length of surface liquid water based on the Beer-Lambert attenuation law.

    Args:
        x:        state vector (liquid water path length, intercept, slope)
        y:        measurement (surface reflectance spectrum)
        wl:       instrument wavelengths
        alpha_lw: wavelength dependent absorption coefficients of liquid water

    Returns:
        resid: residual between modeled and measured surface reflectance
    """

    attenuation = np.exp(-x[0] * 1e7 * alpha_lw)
    rho = (x[1] + x[2] * wl) * attenuation
    resid = rho - y

    return resid

Next we will define a function to retrieve the refractive indices of different water phases provided by the `k_liquid_water_ice.csv` in the `data` folder. We can also preview this data to get a better understanding of the information we are using.

In [ ]:
wp_fp = '../data/k_liquid_water_ice.csv'
k_wi = pd.read_csv(wp_fp)
k_wi.head()

#TODO - Figure out how to do this with hvplot

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(15, 6),  sharex=True, sharey=True, constrained_layout=True)
axs = axs.ravel()
col_n = 0
for i in range(0, 7):
    x = k_wi.iloc[:, col_n+i]
    y = k_wi.iloc[:, col_n+i+1]
    axs[i].scatter(x, y)
    axs[i].set_title(y.name)
    col_n+=1
fig.supylabel('imaginary parts of refractive index')
fig.supxlabel('wavelength')
plt.show()

In [ ]:
# https://github.com/isofit/isofit/blob/main/isofit/core/common.py
def get_refractive_index(k_wi, a, b, col_wvl, col_k):
    """Convert refractive index table entries to numpy array.

    Args:
        k_wi:    variable
        a:       start line
        b:       end line
        col_wvl: wavelength column in pandas table
        col_k:   k column in pandas table

    Returns:
        wvl_arr: array of wavelengths
        k_arr:   array of imaginary parts of refractive index
    """

    wvl_ = []
    k_ = []

    for ii in range(a, b):
        wvl = k_wi.at[ii, col_wvl]
        k = k_wi.at[ii, col_k]
        wvl_.append(wvl)
        k_.append(k)

    wvl_arr = np.asarray(wvl_)
    k_arr = np.asarray(k_)

    return wvl_arr, k_arr

#TODO - Improve this description.

Lastly, we will define a function to fit a state vector...

In [ ]:
# https://github.com/isofit/isofit/blob/main/isofit/inversion/inverse_simple.py
def invert_liquid_water(
    rfl_meas: np.array,
    wl: np.array,
    l_shoulder: float = 850,
    r_shoulder: float = 1100,
    lw_init: tuple = (0.02, 0.3, 0.0002),
    lw_bounds: tuple = ([0, 0.5], [0, 1.0], [-0.0004, 0.0004]),
):
    """Given a reflectance estimate, fit a state vector including liquid water path length
    based on a simple Beer-Lambert surface model.

    Args:
        rfl_meas:   surface reflectance spectrum
        wl:         instrument wavelengths, must be same size as rfl_meas
        l_shoulder: wavelength of left absorption feature shoulder
        r_shoulder: wavelength of right absorption feature shoulder
        lw_init:    initial guess for liquid water path length, intercept, and slope
        lw_bounds:  lower and upper bounds for liquid water path length, intercept, and slope

    Returns:
        solution: estimated liquid water path length, intercept, and slope based on a given surface reflectance
    """

    # params needed for liquid water fitting
    lw_feature_left = np.argmin(abs(l_shoulder - wl))
    lw_feature_right = np.argmin(abs(r_shoulder - wl))
    wl_sel = wl[lw_feature_left : lw_feature_right + 1]

    # load imaginary part of liquid water refractive index and calculate wavelength dependent absorption coefficient
    # __file__ should live at EMIT-Data-Resources/

    data_dir = '../data/' # Hard Coded
    path_k = os.path.join(data_dir, 'k_liquid_water_ice.csv')

    k_wi = pd.read_csv(path_k)
    wl_water, k_water = get_refractive_index(
        k_wi=k_wi, a=0, b=982, col_wvl="wvl_6", col_k="T = 20°C"
    )
    kw = np.interp(x=wl_sel, xp=wl_water, fp=k_water)
    abs_co_w = 4 * np.pi * kw / wl_sel

    rfl_meas_sel = rfl_meas[lw_feature_left : lw_feature_right + 1]

    x_opt = least_squares(
        fun=beer_lambert_model,
        x0=lw_init,
        jac="2-point",
        method="trf",
        bounds=(
            np.array([lw_bounds[ii][0] for ii in range(3)]),
            np.array([lw_bounds[ii][1] for ii in range(3)]),
        ),
        max_nfev=15,
        args=(rfl_meas_sel, wl_sel, abs_co_w),
    )

    solution = x_opt.x

    return solution

### CWC of a Single Pixel

In [ ]:
cwc = invert_liquid_water(point.reflectance.values,point.wavelengths.values)
print(f"CWC for ({point.longitude.values:.3f},{point.latitude.values:.3f}): {cwc[0]:.3f} g/cm^2)")

In [ ]:
# Select Multiple Points 

## CWC Calculation of an ROI

In the previous notebook, we subset our region of interest and exported the file. Since the CWC calculation is computationally intensive, it can take a while to process large scenes, so its more efficient to do this spatial subsetting up front. We can use a function included in the `ewt_calc.py` to calculate the canopy water content on a cropped image, and create a cloud-optimized GeoTIFF (COG) file containing the results.

Set our input filepaths and output directory. 

In [ ]:
fp = '../data/EMIT_L2A_RFL_001_20231014T224006_2328715_002_sedgwick.nc'

In [ ]:
out_dir = '../data/'

Use the `calc_ewt` function to calculate the CWC of the cropped image. This function will also create a COG file containing the CWC results. We can also specify the number of CPUs to use manually with a `n_cpu` argument, or leave it blank to use all but one of the available CPUs.

In [ ]:
calc_ewt(fp, out_dir)

Now we can open the COG created and visualize the results.

In [ ]:
cwc_fp = '../data/EMIT_L2A_RFL_001_20230405T190311_2309513_002_sedgwick.nc'

When opening, we can use `squeeze` to drop the extra `band` dimension.

In [ ]:
ds_cwc = rxr.open_rasterio(cwc_fp).squeeze('band',drop=True)
ds_cwc

In [ ]:
ds_cwc.hvplot.image(x='x',y='y', aspect='equal', cmap='viridis')